In [2]:
# layout_config.py
import plotly.graph_objs as go

# Variáveis para personalizar tamanhos de fontes e tipo de fonte
font_title_size = 18  # Tamanho da fonte do título
font_axis_size = 18   # Tamanho da fonte dos eixos
font_tick_size = 13   # Tamanho da fonte dos ticks
font_legend_size = 18 # Tamanho da fonte da legenda
font_family = 'Arial' # Tipo de fonte
font_color = 'black'  # Cor da fonte

legx = 1  # Posição horizontal da legenda
legy = 1  # Posição vertical da legenda
legxanchor = 'right'  # Alinha a legenda pelo lado direito
legyanchor = 'top'  # Alinha a legenda pelo topo

def layout(title, xtitle, ytitle):
    layout = go.Layout(
        title=dict(
            text=title,
            font=dict(size=font_title_size, family=font_family, color=font_color),
            x=0.5
        ),
        width=800,
        height=600,
        scene=dict(  # <- Adiciona o eixo Z aqui para gráficos 3D
            xaxis=dict(
                title=xtitle,
                range=[0, 1.5],
                titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
                showgrid=True,
                zeroline=True,
                showline=True,
                mirror=True,
                linecolor='black',
                linewidth=2,
                ticks='inside',
                tickwidth=2,
                tickcolor='black',
                tickfont=dict(size=font_tick_size, family=font_family, color=font_color)
            ),
            yaxis=dict(
                title=ytitle,
                range = [0, 1],
                titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
                showgrid=True,
                zeroline=True,
                showline=True,
                mirror=True,
                linecolor='black',
                linewidth=2,
                ticks='inside',
                tickwidth=2,
                tickcolor='black',
                tickfont=dict(size=font_tick_size, family=font_family, color=font_color)
            ),
            zaxis=dict(
                title='ΔP (dP)',
                range=[0, 2e6],
                titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
                tickfont=dict(size=font_tick_size, family=font_family, color=font_color),
                showbackground=True,
                backgroundcolor='white',
                gridcolor='lightgray',
                zeroline=True,
                zerolinecolor='black',
                zerolinewidth=2,
                linecolor='black'
            )
        ),
        plot_bgcolor='white',
        legend=dict(
            x=legx,
            y=legy,
            xanchor=legxanchor,
            yanchor=legyanchor,
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='black',
            borderwidth=1,
            font=dict(size=font_legend_size, family=font_family, color=font_color)
        )
    )
    return layout


In [ ]:
import joblib
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os

cell_size = 12

# ========== Carregar dados experimentais ==========
df = pd.read_excel('Train_data.xlsx', sheet_name='Cs12EXP')
vazao_exp = df.iloc[:, 0].values
dp_exp = df.iloc[:, 3].values
espessura_exp = 1
cell_size_exp = 12
espessura_exp_array = np.full_like(vazao_exp, espessura_exp)

# ========== Carregar modelo e scalers ==========
mlp_model = joblib.load('mlp_model_forch_loocv.pkl')
scalerx = joblib.load('scalerx_mlp_model_forch_loocv.pkl')
scalery = joblib.load('scalery_mlp_model_forch_loocv.pkl')

# ========== Superfície predita ==========
vazao_vals = np.linspace(0, 1, 50)
espessura_vals = np.linspace(1.0, 1.5, 50)
V, E = np.meshgrid(vazao_vals, espessura_vals)
X_input = np.column_stack((V.ravel(), E.ravel(), np.full(V.size, cell_size)))
X_scaled = scalerx.transform(X_input)
Y_scaled = mlp_model.predict(X_scaled).reshape(-1, 1)
Y_pred = scalery.inverse_transform(Y_scaled).reshape(V.shape)

# ========== Carregar dados de treino ==========

# ========== Criar figura ==========
fig = go.Figure()

# Superfície
fig.add_trace(go.Surface(
    z=Y_pred,
    x=V,
    y=E,
    colorscale='Viridis',
    colorbar=dict(
        title='ΔP',
        titlefont=dict(color='black'),  # Define a cor do título da barra de cores
        tickfont=dict(color='black')   # Define a cor da fonte dos ticks da barra de cores
    ),
    opacity=0.8,
    showscale=True
))
# Pontos experimentais
fig.add_trace(go.Scatter3d(
    x=vazao_exp,
    y=espessura_exp_array,
    z=dp_exp,
    mode='markers',
    marker=dict(size=6, color='red', symbol='circle'),
    name='Dados experimentais'
))

# Pontos de treino
df2 = pd.read_excel('Train_data.xlsx', sheet_name='Cs12')
fig.add_trace(go.Scatter3d(
    x=df2.iloc[:, 0].values,
    y=df2.iloc[:, 1].values,
    z=df2.iloc[:, 3].values,
    mode='markers',
    marker=dict(size=6, color='blue', symbol='diamond'),
    name='Dados de treino'
))

# Layout padronizado
fig.update_layout(
    title=dict(
        text='Superfície: ΔP vs Vazão e Espessura (cell size = 8)',
        font=dict(size=font_title_size, family=font_family, color=font_color),
        x=0.5
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=40),
    plot_bgcolor='white',
    scene=dict(
        xaxis=dict(
            title='Vazão',
            titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
            tickfont=dict(size=font_tick_size, family=font_family, color=font_color),
            showbackground=True,
            backgroundcolor='white',
            gridcolor='lightgray',
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,  # Ajuste a espessura da linha zero
            linecolor='black'
        ),
        yaxis=dict(
            title='Espessura',
            titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
            tickfont=dict(size=font_tick_size, family=font_family, color=font_color),
            showbackground=True,
            backgroundcolor='white',
            gridcolor='lightgray',
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,  # Ajuste a espessura da linha zero
            linecolor='black'
        ),
        zaxis=dict(
            title='ΔP (dP)',
            titlefont=dict(size=font_axis_size, family=font_family, color=font_color),
            tickfont=dict(size=font_tick_size, family=font_family, color=font_color),
            showbackground=True,
            backgroundcolor='white',
            gridcolor='lightgray',
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2,  # Ajuste a espessura da linha zero
            linecolor='black'
        )
    ),
    legend=dict(
        x=legx,
        y=legy,
        xanchor=legxanchor,
        yanchor=legyanchor,
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1,
        font=dict(size=font_legend_size, family=font_family, color=font_color)
    )
)

fig.show()
